In [2]:
import pandas as pd
import torch
from torch import nn
import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import sys


In [6]:
def visualization(data):
    if data is None:
        print('data load faile')
        sys.exit()
    for i in range(0,len(data)):
        vis = o3d.visualization.Visualizer()
        obb = data[i].get_oriented_bounding_box()
        vis.create_window()
        vis.add_geometry(obb)
        vis.add_geometry(data[i])
        o3d.visualization.ViewControl.set_zoom(vis.get_view_control(), 0.8)
        vis.run()


    
def plane_detection():
    ###############1
    data = []
    pcd = o3d.io.read_point_cloud("realsensedata.ply")
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                             ransac_n=3,
                                             num_iterations=1000)


    [a, b, c, d] = plane_model
    print(f"Plane equation: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")

    inlier_cloud = pcd.select_by_index(inliers)
    inlier_cloud.paint_uniform_color([1.0, 0, 0])

    outlier_cloud = pcd.select_by_index(inliers, invert=True)

    ###############2
    plane2_model, inliers2 = outlier_cloud.segment_plane(distance_threshold=0.01,
                                                         ransac_n=3,
                                                         num_iterations=1000)
    [a, b, c, d] = plane2_model
    print(f"Plane equation2: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    
    inlier2_cloud = outlier_cloud.select_by_index(inliers2)
    inlier2_cloud.paint_uniform_color([1.0, 0, 0])

    outlier2_cloud = outlier_cloud.select_by_index(inliers2, invert=True)

    ################3
    plane3_model, inliers3 = outlier2_cloud.segment_plane(distance_threshold=0.01,
                                                          ransac_n=3,
                                                          num_iterations=1000)
    [a, b, c, d] = plane3_model
    print(f"Plane equation3: {a:.2f}x + {b:.2f}y + {c:.2f}z + {d:.2f} = 0")
    inlier3_cloud = outlier2_cloud.select_by_index(inliers3)
    inlier3_cloud.paint_uniform_color([1.0, 0, 0])
    
    data.append(inlier_cloud)
    data.append(inlier2_cloud)
    data.append(inlier3_cloud)
    
    p1 = np.asarray(data[0].points)
    p2 = np.asarray(data[1].points)
    p3 = np.asarray(data[2].points)
    
    print(p1)
    print(p2)
    print(p3)

    
    visualization(data)
    

plane_detection()

Plane equation: -0.03x + 0.92y + 0.39z + -0.92 = 0
Plane equation2: -0.39x + -0.62y + 0.68z + -2.22 = 0
Plane equation3: 0.95x + -0.22y + 0.23z + -1.18 = 0
[[0.10253906 0.25424533 1.78417969]
 [0.09667969 0.2542444  1.78417969]
 [0.10253906 0.25488281 1.78256655]
 ...
 [0.84146997 0.61230469 0.99902344]
 [0.84082031 0.61395002 0.99316406]
 [0.64160156 0.58886719 1.03235691]]
[[-1.02246094 -0.67675781  2.06746416]
 [-1.02246094 -0.67261959  2.06542969]
 [-1.02453553 -0.67089844  2.06542969]
 ...
 [ 0.18457031 -0.38226661  2.99121094]
 [ 0.18457031 -0.38378906  2.98943664]
 [ 0.18457031 -0.38378906  2.99693584]]
[[ 0.78808594  0.11085796  1.97167969]
 [ 0.7870562   0.11425781  1.97167969]
 [ 0.78808594  0.11425781  1.96960605]
 ...
 [ 0.99902344 -0.13183594  0.84195958]
 [ 1.00235313 -0.12597656  0.84082031]
 [ 1.00083033 -0.12011719  0.84082031]]


In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import open3d as o3d
import numpy as np


In [14]:
torch.manual_seed(1)

In [15]:
# 훈련 데이터
x1_train = torch.FloatTensor([[73], [93], [89], [96], [73]])
x2_train = torch.FloatTensor([[80], [88], [91], [98], [66]])
x3_train = torch.FloatTensor([[75], [93], [90], [100], [70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])


In [16]:
# 가중치 w와 편향 b 초기화
w1 = torch.zeros(1, requires_grad=True)
w2 = torch.zeros(1, requires_grad=True)
w3 = torch.zeros(1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [17]:
# optimizer 설정
optimizer = optim.SGD([w1, w2, w3, b], lr=1e-5)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} w1: {:.3f} w2: {:.3f} w3: {:.3f} b: {:.3f} Cost: {:.6f}'.format(
            epoch, nb_epochs, w1.item(), w2.item(), w3.item(), b.item(), cost.item()
        ))

Epoch    0/1000 w1: 0.294 w2: 0.294 w3: 0.297 b: 0.003 Cost: 29661.800781
Epoch  100/1000 w1: 0.674 w2: 0.661 w3: 0.676 b: 0.008 Cost: 1.563634
Epoch  200/1000 w1: 0.679 w2: 0.655 w3: 0.677 b: 0.008 Cost: 1.497607
Epoch  300/1000 w1: 0.684 w2: 0.649 w3: 0.677 b: 0.008 Cost: 1.435026
Epoch  400/1000 w1: 0.689 w2: 0.643 w3: 0.678 b: 0.008 Cost: 1.375730
Epoch  500/1000 w1: 0.694 w2: 0.638 w3: 0.678 b: 0.009 Cost: 1.319511
Epoch  600/1000 w1: 0.699 w2: 0.633 w3: 0.679 b: 0.009 Cost: 1.266222
Epoch  700/1000 w1: 0.704 w2: 0.627 w3: 0.679 b: 0.009 Cost: 1.215696
Epoch  800/1000 w1: 0.709 w2: 0.622 w3: 0.679 b: 0.009 Cost: 1.167818
Epoch  900/1000 w1: 0.713 w2: 0.617 w3: 0.680 b: 0.009 Cost: 1.122429
Epoch 1000/1000 w1: 0.718 w2: 0.613 w3: 0.680 b: 0.009 Cost: 1.079378


In [12]:
[Parameter containing:
tensor([[ 0.2975, -0.2548, -0.1119]], requires_grad=True), Parameter containing:
tensor([0.2710], requires_grad=True)]


SyntaxError: invalid syntax (<ipython-input-12-c38780844c59>, line 1)

In [18]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [19]:
print(x_train.shape)
print(y_train.shape)

torch.Size([5, 3])
torch.Size([5, 1])


In [20]:
# 가중치와 편향 선언
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

In [21]:
hypothesis = x_train.matmul(W) + b

In [22]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

# 모델 초기화
W = torch.zeros((3, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 20
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    # 편향 b는 브로드 캐스팅되어 각 샘플에 더해집니다.
    hypothesis = x_train.matmul(W) + b

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
        epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
    ))


Epoch    0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch    1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch    2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch    3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch    4/20 hypothesis: tensor([137.7968, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936005
Epoch    5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371010
Epoch    6/20 hypothesis: tensor([148.1035, 178.0144, 175.3980, 191.0042, 135.7812]) Cost: 29.758139
Epoch    7/20 hypothesis: tensor([150.1744, 180.5042, 177.8508, 193.6753, 137.6805]) Cost: 10.445305
Epoch    8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391228
Epoch    9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493135
Epoch   10/20 hypo